# 0 Install package

In [ ]:
!pip install streamlit pandas geopandas folium branca

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.9 MB/s eta 0:00:00


In [ ]:
!pip install streamlit_folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.4/328.4 kB 2.6 MB/s eta 0:00:00


In [ ]:
import requests
import pandas as pd
import plotly.graph_objects as go
from sqlalchemy import create_engine
import folium
from folium.plugins import MarkerCluster

Education Level

Income Level

Employment

House

# 2 Greate map


In [ ]:
!pip install pyshp simpledbf

  Preparing metadata (setup.py) ... done
  Created wheel for simpledbf: filename=simpledbf-0.2.6-py3-none-any.whl size=13786 sha256=68e9185b8443dd219895cbd7042f80411f2e0b4dbda80c4d975169878c0c2677
  Stored in directory: /root/.cache/pip/wheels/37/52/21/14be45b7c160488637e82d6a317f4379458bb4dd60be21d5fa
Successfully built simpledbf


In [ ]:
!pip install streamlit

In [ ]:
!pip install pyngrok

In [ ]:
!pip install openai

In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd
import geopandas as gpd
import folium
from folium import Choropleth
import json
import streamlit.components.v1 as components
from streamlit_folium import st_folium

# Load your datasets here
data = pd.read_csv("/content/indexes_dataset.csv")
manual_mapping = pd.read_csv("/content/Completed_FineNeighborhood_to_CommunityAreaName.csv")
metadata = pd.read_csv("/content/indexes_metadata.csv")

# Load local GeoJSON file
with open("/content/chicago_neighborhoods.geojson", "r", encoding="utf-8") as f:
    geojson_data = json.load(f)

# Index options and their factors
index_factors = {
    "wellness_index": ["EVI","SSI","EOI","CARDIO", "DIABETES", "RESPIR", "MHDIS", "CURSMOKE", "BINGEDRINK", "DRUGOD", "PHYINACT"],
    "EVI": ["AIRQUAL", "WATQUAL", "HEATVUL", "NOISEP"],
    "SSI": ["SCHLRAT", "HSDROP", "UNEMPLOY", "MEDINC"],
    "EOI": ["HOUSPRB", "HOUSBURD", "BROADND", "TRAFVOL", "COLLACC"]
}

# Sidebar selections
st.set_page_config(layout="wide")
st.sidebar.title("Index Configuration")
selected_index = st.sidebar.selectbox("Select Index", list(index_factors.keys()))
st.sidebar.subheader("Index Description")
index_description = metadata[metadata['index_name'] == selected_index]['description'].values
st.markdown(
    """
    <style>
        [data-testid="stSidebar"] {
            min-width: 400px;
            max-width: 400px;
        }
    </style>
    """,
    unsafe_allow_html=True
)

with st.sidebar.expander("ℹ️ Index Description", expanded=False):
    if len(index_description):
        st.markdown(
            f"<div style='font-size: 12px; text-align: justify;'>{index_description[0]}</div>",
            unsafe_allow_html=True
        )
    else:
        st.markdown("_No index-level description found._")

selected_factors = index_factors[selected_index]

with st.sidebar.expander("🧪 Factor Descriptions", expanded=True):
    rows = ""
    for factor in selected_factors:
        desc_row = metadata[metadata['index_name'] == factor]['description'].values
        desc_text = desc_row[0] if len(desc_row) else "(No description found)"
        rows += f"<tr><td><strong>{factor}</strong></td><td>{desc_text}</td></tr>"

    table_html = f"""
    <div style='font-size: 11px;text-align: justify;'>
        <table style='width: 100%; border-collapse: collapse;'>
            {rows}
        </table>
    </div>
    """
    st.markdown(table_html, unsafe_allow_html=True)

# Load GeoDataFrame
gdf_raw = gpd.GeoDataFrame.from_features(geojson_data["features"])
gdf_raw = gdf_raw.rename(columns={"pri_neigh": "FineNeighborhood"})
gdf_raw.set_crs(epsg=4326, inplace=True)

# Merge with manual mapping to get CommunityAreaName
gdf = gdf_raw.merge(manual_mapping, on="FineNeighborhood", how="left")

# Merge with index data
gdf = gdf.merge(data, on="CommunityAreaName", how="left")

# Ensure the CommunityAreaName exists in GeoJSON for rendering
gdf["CommunityAreaName"] = gdf["CommunityAreaName"].fillna("Unknown")

# Function to render map
def render_map(gdf, column, title):
    m = folium.Map(location=[41.8781, -87.6298], zoom_start=10,use_container_width=True)

    # Workaround: add CommunityAreaName to each feature property before converting to JSON
    gdf_json = json.loads(gdf.to_json())
    for idx, feature in enumerate(gdf_json["features"]):
        feature["properties"]["CommunityAreaName"] = gdf.iloc[idx]["CommunityAreaName"]

    Choropleth(
        geo_data=gdf_json,
        name="choropleth",
        data=gdf,
        columns=["CommunityAreaName", column],
        key_on="feature.properties.CommunityAreaName",
        fill_color="YlOrRd",
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=title,
        highlight=True,
        nan_fill_color='gray'
    ).add_to(m)

    def style_function(feature):
        return {'fillColor': 'transparent','color': 'black','weight': 1,'fillOpacity': 0}

    def highlight_function(feature):
        return {'fillColor': '#ffff00','color': 'black','weight': 2,'fillOpacity': 0.3}

    tooltip_data = []
    for _, row in gdf.iterrows():
        name = row["CommunityAreaName"]
        value = row[column]
        value_str = f"{value:.2f}" if pd.notna(value) else "No data"
        tooltip_data.append({"tooltip": f"{name}  {value_str}"})

    tooltip_gdf = gdf.copy()
    tooltip_gdf["tooltip"] = [d["tooltip"] for d in tooltip_data]
    tooltip_gdf.set_crs(epsg=4326, inplace=True)

    folium.GeoJson(
        tooltip_gdf,
        name="Neighborhood Borders",
        style_function=style_function,
        highlight_function=highlight_function,
        tooltip=folium.GeoJsonTooltip(fields=["tooltip"], aliases=["Community: "])
    ).add_to(m)

    return m


# Main layout
st.title("Chicago Neighborhood Dashboard")
st.markdown(f"### Main Map: {selected_index}")
main_map = render_map(gdf, selected_index, selected_index)
st_folium(main_map, use_container_width=True)

# Panel maps with factor selectors on top
col1, col2 = st.columns(2)
with col1:
    st.markdown("<h4 style='text-align: center;'>Panel 1</h4>", unsafe_allow_html=True)
    st.markdown("<div style='font-size: 13px; color: gray;text-align: center;'>Select Panel 1 Factor</div>", unsafe_allow_html=True)
    factor1 = st.selectbox("", selected_factors, key="panel1")
    st.markdown(f"#### {factor1}")
    panel1_map_html = render_map(gdf, factor1, factor1)
    st_folium(panel1_map_html, use_container_width=True)

with col2:
    st.markdown("<h4 style='text-align: center;'>Panel 2</h4>", unsafe_allow_html=True)
    st.markdown("<div style='font-size: 13px; color: gray;text-align: center;'>Select Panel 2 Factor</div>", unsafe_allow_html=True)
    remaining_factors = [f for f in selected_factors if f != st.session_state.panel1]
    factor2 = st.selectbox("", remaining_factors, key="panel2")
    st.markdown(f"#### {factor2}")
    panel2_map_html = render_map(gdf, factor2, factor2)
    st_folium(panel2_map_html, use_container_width=True)

Overwriting app.py


In [ ]:
# 设置 ngrok authtoken
from pyngrok import ngrok
ngrok.set_auth_token("2vh8RdP8gH4DZ4IPG49jCTHtQkD_8766JsxXk3kHKzYxUJxY6")

# 启动 ngrok 隧道映射本地 8501 端口
public_url = ngrok.connect("8501")
print("Streamlit app public URL:", public_url)

Streamlit app public URL: NgrokTunnel: "https://d911-34-19-88-1.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
# 在后台启动 Streamlit 应用
!streamlit run app.py &




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.19.88.1:8501

2025-04-13 23:25:59.264 `label` got an empty value. This is discouraged for accessibility reasons and may be disallowed in the future by raising an exception. Please provide a non-empty label and hide it with label_visibility if needed.
2025-04-13 23:26:02.272 `label` got an empty value. This is discouraged for accessibility reasons and may be disallowed in the future by raising an exception. Please provide a non-empty label and hide it with label_visibility if needed.
2025-04-13 23:26:13.048 `label` got an empty value. This is discouraged for accessibility reasons and may be disallowed in the future by raising an exception. Please provide a non-empty label and hide it with label_visibility if needed.
2025-04-13 23:26:18.825 `label` got an empty value. This is discouraged for accessibility reasons and may be disallo